![Cython](figures/Cython_logo.svg)

Cython is a bit like numba.
Let's start with an example.

In [ ]:
# We start with our optimized Python implementation.
# Parameter 'f' has been renamed to 'func', to distinguish it from the 'f' function.

def f(x):
    return x**4 - 3 * x

def integrate_f(func, a, b, n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [ ]:
# We increase the number of points (from 3k to 1M) for easier measurement.

integrate_f(f, -10, +10, 1_000_000)

In [ ]:
%timeit integrate_f(f, -10, +10, 1_000_000)

Our first step is to load the `cython` extension and apply `%%cython` magic to the whole cell.

`f`, `integrate_f` are renamed to `f2`, `integrate_f2`.

The code is otherwise unchanged.

In [ ]:
%load_ext cython

In [ ]:
%%cython

def f2(x):
    return x**4 - 3 * x

def integrate_f2(func, a, b, n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [ ]:
integrate_f2(f2, -10, +10, 1_000_000)

In [ ]:
%timeit integrate_f2(f2, -10, +10, 1_000_000)

In [ ]:
# f2 and integrate_f2 are now compiled functions

f2, integrate_f2

Our next step is to set types of the function parameters.

This is called "type specialiation".

Our code is not valid Python anymore, only Cython can understand it.

In [ ]:
%%cython

def f3(double x):
    return x**4 - 3 * x

def integrate_f3(func, double a, double b, int n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [ ]:
integrate_f3(f3, -10, +10, 1_000_000)

In [ ]:
%timeit integrate_f3(f3, -10, +10, 1_000_000)

Our next step is to also provide types of internal variables.

We also use `-a` to tell `%%cython` to generate annotated output with hints about Python ↔ C-code interation.

In [ ]:
%%cython -a

def f4(double x):
    return x**4 - 3 * x

def integrate_f4(func, double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [ ]:
integrate_f4(f4, -10, +10, 1_000_000)

In [ ]:
%timeit integrate_f4(f4, -10, +10, 1_000_000)

## Exercise 01-cython-primes

Please open [01-cython-primes/exercise.ipynb](01-cython-primes/exercise.ipynb) and follow instructions therein.

### Cython function type specialization

We see strong yellow in the internal loop:
every time we call `f`, which is a Python function, conversion from Python types to C types and back must be done. 

This is slow.

In [ ]:
%%cython

# We do "function specialization", by changing 'f5' to be a C function.
#
# On the first attempt we get a warning, because the generated C function is not used.

cdef double f5(double x):
    return x**4 - 3 * x

def integrate_f5(func, double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [ ]:
%%cython -a

# In the second attempt, integrate_f5 hardcodes the function to call.
# We can rename it to just 'f', because it'll not be visible anywhere.

cdef double f(double x):
    return x**4 - 3 * x

def integrate_f5(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += f(a + (i + 0.5) * dx) * dx
    return s

In [ ]:
integrate_f5(-10, +10, 1_000_000)

In [ ]:
%timeit integrate_f5(-10, +10, 1_000_000)

### Cython formula optimization

As a final optimization step, we can replace the call to `x**4`,
which generates a call to the C function `pow()`, with a series of multiplications.

We can also reorder the expression a bit to save one multiplication.

In [ ]:
%%cython

cdef double f(double x):
    return (x*x*x - 3) * x

def integrate_f6(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += f(a + (i + 0.5) * dx) * dx
    return s

In [ ]:
integrate_f6(-10, +10, 1_000_000)

In [ ]:
%timeit integrate_f6(-10, +10, 1_000_000)

## Exercise: 02-cython-distrib

 Please open a terminal, `cd` to `02-cython-distrib/`, and follow the instructions:

In [ ]:
from IPython import display
display.display(display.Markdown(open('02-cython-distrib/README.md').read()))


# Cython and Numpy Arrays

## Array summing

Let's start by summing up an array.

`const double [:]` means that the parameter is a immutable 1D memoryview.

We can use this function on numpy arrays.

In [ ]:
%%cython -a

import numpy as np

def mysum(const double [:] A):
    cdef:
        double s = 0.0
        ssize_t n = A.shape[A.ndim - 1]
        ssize_t i
    
    for i in range(n):
        s += A[i]
    return s

In [ ]:
x = np.linspace(0, 10, 54)
mysum(x)

In [ ]:
%%cython -a

# From the yellow output above, we see that in line 11 a check
# is performed to optionally throw BufferIndexError.
# We know that this cannot happen, so we can tell Cython to
# disable the check with 'boundscheck(False)'.
#
# While at it, let's disable support for negative indices with
# 'wraparound(False)'. We don't use those.

import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
def mysum(const double [:] A):
    cdef:
        double s = 0.0
        ssize_t n = A.shape[A.ndim - 1]
        ssize_t i
    
    for i in range(n):
        s += A[i]
    return s

## Exercise 03-cython-mean3filter

Please open [03-cython-mean3filter/exercise.ipynb](03-cython-mean3filter/exercise.ipynb) and follow instructions therein.

### Summary: why cython?

Cython is clearly more work, but:

- it is less magic, so you can really understand what is going on
- you have control over the details
- is it very old (1st pyrex release April 2002) and widespread
- very portable — the compiled .so file or the generated .c file can be distributed
- Cython speed is close to C
- Cython cdef functions can call C/C++ code — Cython is great for wrapping external libraries

When to use Cython:

- number-crunching functions that cannot be vectorized with Numpy or other librararies
- functions with complicated internal logic of conditions
- code which needs to call library in C/C++/Fortan/other compiled code
- to "wrap" a library in C/C++/Fortan/other compiled code to make it useful from Python
  (esp. when numpy arrays are used)
  
When to use Numba instead:
- in general numba is easier, so try it first
- use cython when numba is not appropriate (because it doesn't work, or you need to distribute the compiled code, or you need to wrap external libraries)
